In [1]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import getpass
from IPython.display import display, Image
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dve-shm-uage')
options.add_argument('ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors=yes')

In [3]:
def show_screen():
    bot.save_screenshot("out.png")
    display(Image("out.png"))

In [4]:
username = 'bensshieh4333@gmail.com'
password = 'Ben23340020`'

#### 1. Login Page

In [5]:
def page_login():
    url = 'https://www.cdp.net/en/users/sign_in'
    bot = webdriver.Chrome(options=options)
    wait = WebDriverWait(bot, 20)
    bot.get(url)
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#onetrust-accept-btn-handler"))).click()
    time.sleep(1)

    bot.find_element(By.ID, 'user_email').send_keys(username)
    bot.find_element(By.ID, 'user_password').send_keys(password)
    button = bot.find_element(By.CSS_SELECTOR, "button[type='submit']")
    button.click()
    time.sleep(1)

    bot.get('https://www.cdp.net/en/scores')

    popup_button = bot.find_element(By.CLASS_NAME, "ga-track-regional-popup-item-north-america")
    popup_button.click()
    time.sleep(1)

    return bot

#### 2. Search Company

In [6]:
def search_company(bot, company_name):
    input_box = bot.find_element(By.ID, "queries_name")
    input_box.send_keys(company_name)
    submit_button = bot.find_element(By.CLASS_NAME, "response_search_form--btn")
    submit_button.click()

    link = bot.find_element(By.CSS_SELECTOR, "a.pagination__per_page--selected")
    href = link.get_attribute('href')
    href = href.replace('page=5', 'page=20')
    bot.get(href)

    return bot

#### 3. Search Data

In [7]:
def scraper(bot, year):
    # Find the table by its class name (or other suitable selector)
    table = bot.find_element(By.CLASS_NAME, "sortable_table")

    # Extract all rows from the table
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Initialize lists to store data and hrefs
    data = []
    hrefs = []

    # Extract data from each row
    for row in rows[1:]:  # Skip the header row
        cols = row.find_elements(By.TAG_NAME, "td")
        row_data = []
        row_hrefs = []
        for i, col in enumerate(cols):
            links = col.find_elements(By.TAG_NAME, "a")
            if links:
                # Extract text and href for each link
                link_text = links[0].text
                link_href = links[0].get_attribute('href')
                row_data.append(link_text)
                row_hrefs.append(link_href)
            else:
                # If no link, add the text and a None for href
                row_data.append(col.text)
                row_hrefs.append(None)
        
        data.append(row_data)
        hrefs.append(row_hrefs)

    # Create a DataFrame from the extracted data
    column_headers = [col.text for col in rows[0].find_elements(By.TAG_NAME, "th")]
    df = pd.DataFrame(data, columns=column_headers)

    # Add the hrefs as a new column
    df['Response Links'] = pd.Series([href[1] for href in hrefs])

    df = df[df['Year'] == year]

    return df
    

In [8]:
def find_section(row):

    if row['Response'] == 'Climate Change 2023':
        bot.get(row['Response Links'])
        WebDriverWait(bot, 10).until(EC.visibility_of_element_located((By.ID, 'formatted_responses_section_31715')))
        scraped_text = bot.find_element(By.ID, 'formatted_responses_section_31715').text
    elif row['Response'] == 'Forests 2023':
        bot.get(row['Response Links'])
        WebDriverWait(bot, 10).until(EC.visibility_of_element_located((By.ID, 'formatted_responses_section_90809')))
        scraped_text = bot.find_element(By.ID, 'formatted_responses_section_90809').text    
    elif row['Response'] == 'Water Security 2023':
        bot.get(row['Response Links'])
        WebDriverWait(bot, 10).until(EC.visibility_of_element_located((By.ID, 'formatted_responses_section_31639')))
        scraped_text = bot.find_element(By.ID, 'formatted_responses_section_31639').text   

    else:
        scraped_text = 'Unknown Response Type'

    return scraped_text

    
def apply_scraping(df):
    global bot

    df['text'] = df.apply(find_section, axis=1)

    bot.quit()
    return df 

In [9]:
company_name = 'General Mills Inc.'
year = '2023'
bot = page_login() # Login to the website
bot = search_company(bot, company_name) # Enter company name
df = scraper(bot, year) # Specify the year
df_updated = apply_scraping(df)
df_updated

,Name,Response,Year,Status,Score,Response Links,text
0,General Mills Inc.,Climate Change 2023,2023,Submitted,Not Scored,https://www.cdp.net/en/cdp2/redirect?campaign_...,C4.3b\n(C4.3b) Provide details on the initiati...
1,General Mills Inc.,Forests 2023,2023,Submitted,Not Scored,https://www.cdp.net/en/cdp2/redirect?campaign_...,F6.11\n(F6.11) Do you participate in any other...
2,General Mills Inc.,Water Security 2023,2023,Submitted,Not Scored,https://www.cdp.net/en/cdp2/redirect?campaign_...,W8.1b\n(W8.1b) Provide details of your water-r...
